In [ ]:
import logging
import sys
import pathlib
from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
import tifffile
import matplotlib.pyplot as plt
import staintools

sys.path.append('../src')
import settings
import annotation_utils

In [ ]:
# Raw training and test data
df_hubmap_hpa_organ_segmentation_metadata = pd.concat((
    pd.read_csv(settings.DATA / 'train_metadata.csv'),
    pd.read_csv(settings.DATA / 'test_metadata.csv')
), axis=0, ignore_index=True)

# HPA dictionary data
df_hpa_metadata = pd.read_csv(settings.DATA / 'external_data' / 'HPA' / 'metadata.csv')
# HuBMAP Colonic Crypt data
df_hubmap_colonic_crypt_metadata = pd.read_csv(settings.DATA / 'external_data' / 'HuBMAP_Colonic_Crypt' / 'metadata.csv')

In [ ]:
def visualize_stain_transform(raw_image, transformed_image, title):

    """
    Visualize raw and transformed_image 

    Parameters
    ----------
    raw_image (numpy.ndarray of shape (height, width, 3)): Raw image array
    transformed_image (numpy.ndarray of shape (height, width, 3)): Transformed image array
    title (str): Title of the visualization
    """

    fig, axes = plt.subplots(figsize=(32, 20), ncols=2)
    axes[0].imshow(raw_image)
    axes[1].imshow(transformed_image)
    for i in range(2):
        axes[i].set_xlabel('')
        axes[i].set_ylabel('')
        axes[i].tick_params(axis='x', labelsize=15, pad=10)
        axes[i].tick_params(axis='y', labelsize=15, pad=10)
        
    axes[0].set_title(f'Raw Image\nMean: {np.mean(raw_image):2f} - Std: {np.std(raw_image):2f}\nMin: {np.min(raw_image)} - Max: {np.max(raw_image)}', size=25, pad=15)
    axes[1].set_title(f'Transformed Image\nMean: {np.mean(transformed_image):2f} - Std: {np.std(transformed_image):2f}\nMin: {np.min(transformed_image)} - Max: {np.max(transformed_image)}', size=25, pad=15)
    fig.suptitle(title, fontsize=50)
    
    plt.show()


In [ ]:
image1 = tifffile.imread(df_hubmap_hpa_organ_segmentation_metadata.loc[2, 'image_filename'])
image2 = tifffile.imread(df_hubmap_hpa_organ_segmentation_metadata.loc[351, 'image_filename'])

## Luminosity Standardizer

In [ ]:
image1_luminosity_standardized = staintools.LuminosityStandardizer.standardize(image1)
visualize_stain_transform(
    raw_image=image1,
    transformed_image=image1_luminosity_standardized,
    title='Luminosity Standardizer'
)

image2_luminosity_standardized = staintools.LuminosityStandardizer.standardize(image2)
visualize_stain_transform(
    raw_image=image2,
    transformed_image=image2_luminosity_standardized,
    title='Luminosity Standardizer'
)

## Reinhard Color Normalizer

In [ ]:
reinhard_color_normalizer = staintools.ReinhardColorNormalizer()

reinhard_color_normalizer.fit(image1_luminosity_standardized)
image2_color_normalized = reinhard_color_normalizer.transform(image2_luminosity_standardized)
visualize_stain_transform(
    raw_image=image2,
    transformed_image=image2_color_normalized,
    title='Reinhard Color Normalizer'
)

reinhard_color_normalizer.fit(image2_luminosity_standardized)
image1_color_normalized = reinhard_color_normalizer.transform(image1_luminosity_standardized)
visualize_stain_transform(
    raw_image=image1,
    transformed_image=image1_color_normalized,
    title='Reinhard Color Normalizer'
)

## Stain Normalizer

In [ ]:
vahadane_normalizer = staintools.StainNormalizer(method='vahadane')
vahadane_normalizer.fit(image1_luminosity_standardized)
image2_normalized = vahadane_normalizer.transform(image2_luminosity_standardized)

visualize_stain_transform(
    raw_image=image2,
    transformed_image=image2_normalized,
    title='Vahadane Normalizer'
)

vahadane_normalizer.fit(image2_luminosity_standardized)
image1_normalized = vahadane_normalizer.transform(image1_luminosity_standardized)

visualize_stain_transform(
    raw_image=image1,
    transformed_image=image1_normalized,
    title='Vahadane Normalizer'
)

In [ ]:
macenko_normalizer = staintools.StainNormalizer(method='macenko')
macenko_normalizer.fit(image1_luminosity_standardized)
image2_normalized = macenko_normalizer.transform(image2_luminosity_standardized)

visualize_stain_transform(
    raw_image=image2,
    transformed_image=image2_normalized,
    title='Macenko Normalizer'
)

macenko_normalizer.fit(image2_luminosity_standardized)
image1_normalized = macenko_normalizer.transform(image1_luminosity_standardized)

visualize_stain_transform(
    raw_image=image1,
    transformed_image=image1_normalized,
    title='Macenko Normalizer'
)

## Stain Augmentor

In [ ]:
vahadane_augmentor = staintools.StainAugmentor(method='vahadane', sigma1=0.5, sigma2=0.5, augment_background=False)
vahadane_augmentor.fit(image1_luminosity_standardized)

visualize_stain_transform(
    raw_image=image1_luminosity_standardized,
    transformed_image=np.uint8(vahadane_augmentor.pop()),
    title='Vahadane Augmentor'
)

vahadane_augmentor.fit(image2_luminosity_standardized)

visualize_stain_transform(
    raw_image=image2_luminosity_standardized,
    transformed_image=np.uint8(vahadane_augmentor.pop()),
    title='Vahadane Augmentor'
)

In [ ]:
macenko_augmentor = staintools.StainAugmentor(method='macenko', sigma1=0.5, sigma2=0.5, augment_background=False)
macenko_augmentor.fit(image1_luminosity_standardized)

visualize_stain_transform(
    raw_image=image1_luminosity_standardized,
    transformed_image=np.uint8(macenko_augmentor.pop()),
    title='Macenko Augmentor Augmentor'
)

macenko_augmentor.fit(image2_luminosity_standardized)

visualize_stain_transform(
    raw_image=image2_luminosity_standardized,
    transformed_image=np.uint8(macenko_augmentor.pop()),
    title='Macenko Augmentor'
)